# Dimensionality Reduction - Data Science Koans

Welcome to Notebook 11: Dimensionality Reduction!

## What You Will Learn
- Principal Component Analysis (PCA) fundamentals
- Choosing optimal number of components
- Interpreting explained variance and feature loadings
- t-SNE for non-linear visualization
- Understanding the curse of dimensionality
- When and how to apply dimensionality reduction

## Why This Matters
As datasets grow larger and more complex, dimensionality reduction becomes crucial for:
- **Visualization**: Project high-D data to 2D/3D for human understanding
- **Performance**: Reduce computational cost and memory usage
- **Noise Reduction**: Remove irrelevant features and focus on signal
- **Storage**: Compress data while preserving important patterns
- **Analysis**: Overcome the curse of dimensionality

## Prerequisites
- Clustering basics (Notebook 10)
- Understanding of linear algebra concepts
- Experience with scikit-learn

## How to Use
1. Read each koan's objective and explanation
2. Complete the TODO sections with your implementation
3. Run the validation to check correctness
4. Learn from any errors and iterate
5. Move to next koan when passing

Let's reduce some dimensions!

In [ ]:
# Setup - Run first!
import sys
sys.path.append('../..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits, load_iris, load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import pdist

from koans.core.validator import KoanValidator
from koans.core.progress import ProgressTracker

validator = KoanValidator("11_dimensionality_reduction")
tracker = ProgressTracker()

print("Setup complete!")
print(f"Current progress: {tracker.get_notebook_progress('11_dimensionality_reduction')}%")

## KOAN 11.1: Principal Component Analysis
**Objective**: Reduce Iris dataset to 2D using PCA  
**Difficulty**: Intermediate-Advanced

PCA finds the directions (principal components) of maximum variance in your data. 
The first principal component captures the most variance, the second captures the most remaining variance orthogonal to the first, and so on.

**Key Concept**: PCA is a linear transformation that projects data onto a lower-dimensional subspace while preserving as much variance as possible.

In [ ]:
def apply_pca_to_iris():
    """
    Apply PCA to reduce the Iris dataset from 4D to 2D.
    
    Steps:
    1. Load the iris dataset
    2. Get the feature matrix (iris.data)
    3. Create PCA with 2 components
    4. Fit and transform the data
    5. Return the 2D transformed data
    """
    iris = load_iris()
    X = iris.data
    
    # TODO: Create PCA instance with n_components=2
    pca = None
    
    # TODO: Fit PCA and transform data to 2D
    X_2d = None
    
    return X_2d

@validator.koan(1, "Principal Component Analysis", difficulty="Intermediate-Advanced")
def validate():
    result = apply_pca_to_iris()
    
    # Verify we got the right shape
    assert result is not None, "Function returned None - did you forget to implement it?"
    assert isinstance(result, np.ndarray), "Result should be a numpy array"
    assert result.shape == (150, 2), f"Expected shape (150, 2), got {result.shape}"
    
    # Verify it's actually a meaningful transformation
    assert not np.allclose(result, 0), "Result is all zeros - check your PCA implementation"
    
    print("✓ Successfully reduced Iris from 4D to 2D!")
    print(f"  - Original shape: (150, 4)")
    print(f"  - Reduced shape: {result.shape}")

validate()

## KOAN 11.2: Explained Variance
**Objective**: Calculate cumulative explained variance ratios  
**Difficulty**: Intermediate-Advanced

When we reduce dimensions, we lose some information. The explained variance ratio tells us how much of the original variance each component captures.

**Key Concept**: `explained_variance_ratio_` shows the percentage of total dataset variance explained by each principal component. This helps us understand how much information we retain.

In [ ]:
def analyze_digits_variance():
    """
    Apply PCA to digits dataset and return cumulative explained variance.
    
    The digits dataset has 64 features (8x8 pixel images).
    We'll see how much variance the first 10 components capture.
    
    Returns:
        np.ndarray: Cumulative explained variance ratios for first 10 components
    """
    digits = load_digits()
    X = digits.data  # Shape: (1797, 64)
    
    # TODO: Create PCA with 10 components and fit to the digits data
    pca = None
    
    # TODO: Get explained variance ratios and compute cumulative sum
    # Hint: Use np.cumsum() on pca.explained_variance_ratio_
    cumulative_variance = None
    
    return cumulative_variance

@validator.koan(2, "Explained Variance", difficulty="Intermediate-Advanced") 
def validate():
    result = analyze_digits_variance()
    
    assert result is not None, "Function returned None"
    assert isinstance(result, np.ndarray), "Result should be a numpy array"
    assert len(result) == 10, f"Expected 10 values, got {len(result)}"
    
    # Check that it's cumulative (monotonically increasing)
    assert np.all(np.diff(result) >= 0), "Cumulative variance should be increasing"
    
    # Should be between 0 and 1
    assert np.all(result >= 0) and np.all(result <= 1), "Variance ratios should be between 0 and 1"
    
    # First 10 components should explain significant variance
    assert result[-1] > 0.5, f"First 10 components should explain >50% variance, got {result[-1]*100:.1f}%"
    
    print(f"✓ First 10 components explain {result[-1]*100:.1f}% of variance")
    print(f"  - 1st component: {result[0]*100:.1f}%")
    print(f"  - 2nd component: {result[1]*100:.1f}%") 
    print(f"  - 10th component: {result[-1]*100:.1f}%")

validate()

## KOAN 11.3: Scree Plot and Component Selection
**Objective**: Determine optimal number of components for 95% variance  
**Difficulty**: Intermediate-Advanced

A common question in PCA is: "How many components should I keep?" One approach is to choose enough components to retain a specific percentage (like 95%) of the original variance.

**Key Concept**: The "elbow method" looks for the point where adding more components gives diminishing returns in explained variance.

In [ ]:
def find_components_for_variance_threshold():
    """
    Find minimum number of components needed to explain 95% of variance.
    
    We'll use more components (up to 50) to find the exact cutoff.
    
    Returns:
        int: Number of components needed for 95% explained variance
    """
    digits = load_digits()
    X = digits.data
    
    # TODO: Create PCA with 50 components and fit to digits data
    pca = None
    
    # TODO: Calculate cumulative explained variance
    cumulative_variance = None
    
    # TODO: Find first index where cumulative variance >= 0.95
    # Hint: Use np.argmax(cumulative_variance >= 0.95) and add 1
    n_components_95 = None
    
    return n_components_95

@validator.koan(3, "Scree Plot and Component Selection", difficulty="Intermediate-Advanced")
def validate():
    result = find_components_for_variance_threshold()
    
    assert result is not None, "Function returned None"
    assert isinstance(result, (int, np.integer)), "Result should be an integer"
    assert 1 <= result <= 50, f"Expected 1-50 components, got {result}"
    
    # Verify the result by checking it actually gives >= 95% variance
    digits = load_digits()
    pca_check = PCA(n_components=result).fit(digits.data)
    variance_check = np.sum(pca_check.explained_variance_ratio_)
    assert variance_check >= 0.95, f"Components {result} only explain {variance_check*100:.1f}% variance"
    
    print(f"✓ Need {result} components to explain ≥95% variance")
    print(f"  - Actual variance explained: {variance_check*100:.2f}%")
    print(f"  - Dimension reduction: 64 → {result} ({(1-result/64)*100:.1f}% reduction)")

validate()

## KOAN 11.4: PCA for Visualization  
**Objective**: Project digits dataset to 2D for visualization  
**Difficulty**: Intermediate-Advanced

One of the most powerful uses of PCA is visualizing high-dimensional data. By projecting to 2D or 3D, we can plot data and look for patterns, clusters, or outliers.

**Key Concept**: While we lose information going to 2D, we gain the ability to visualize and intuitively understand our data structure.

In [ ]:
def project_digits_for_visualization():
    """
    Project the digits dataset to 2D for visualization purposes.
    
    Returns:
        tuple: (X_2d, y) where X_2d is the 2D projection and y are the digit labels
    """
    digits = load_digits()
    X = digits.data    # Shape: (1797, 64) - flattened 8x8 images
    y = digits.target  # Shape: (1797,) - digit labels 0-9
    
    # TODO: Create PCA with 2 components
    pca = None
    
    # TODO: Fit PCA and transform data to 2D
    X_2d = None
    
    return X_2d, y

@validator.koan(4, "PCA for Visualization", difficulty="Intermediate-Advanced")
def validate():
    X_2d, y = project_digits_for_visualization()
    
    assert X_2d is not None and y is not None, "Function returned None values"
    assert isinstance(X_2d, np.ndarray), "X_2d should be numpy array"
    assert isinstance(y, np.ndarray), "y should be numpy array"
    assert X_2d.shape == (1797, 2), f"Expected X_2d shape (1797, 2), got {X_2d.shape}"
    assert len(y) == 1797, f"Expected y length 1797, got {len(y)}"
    assert set(np.unique(y)) == set(range(10)), "y should contain digits 0-9"
    
    # Check that the projection has reasonable spread
    assert np.std(X_2d[:, 0]) > 1, "First component should have reasonable variance"
    assert np.std(X_2d[:, 1]) > 1, "Second component should have reasonable variance"
    
    print("✓ Successfully projected 64D digits to 2D!")
    print(f"  - Original: {digits.data.shape}")
    print(f"  - Projected: {X_2d.shape}")
    print(f"  - PC1 range: [{X_2d[:, 0].min():.1f}, {X_2d[:, 0].max():.1f}]")
    print(f"  - PC2 range: [{X_2d[:, 1].min():.1f}, {X_2d[:, 1].max():.1f}]")
    
    # Calculate variance explained by first 2 components
    digits = load_digits()
    pca_check = PCA(n_components=2).fit(digits.data)
    variance_2d = np.sum(pca_check.explained_variance_ratio_)
    print(f"  - Variance retained: {variance_2d*100:.1f}%")

validate()

## KOAN 11.5: Feature Loadings
**Objective**: Interpret PCA components through feature loadings  
**Difficulty**: Intermediate-Advanced

Feature loadings tell us which original features contribute most to each principal component. This helps us understand what each PC represents in terms of the original variables.

**Key Concept**: `pca.components_` contains the loadings. Each row is a PC, each column is an original feature. Large absolute values indicate high importance.

In [ ]:
def get_iris_pca_loadings():
    """
    Fit PCA to Iris dataset and return the feature loadings (components).
    
    Iris features are: sepal_length, sepal_width, petal_length, petal_width
    We'll see which features contribute most to each of the first 2 components.
    
    Returns:
        np.ndarray: PCA components/loadings matrix, shape (2, 4)
    """
    iris = load_iris()
    X = iris.data
    
    # TODO: Create PCA with 2 components and fit to iris data  
    pca = None
    
    # TODO: Return the components (loadings) matrix
    # Hint: Use pca.components_
    loadings = None
    
    return loadings

@validator.koan(5, "Feature Loadings", difficulty="Intermediate-Advanced")
def validate():
    result = get_iris_pca_loadings()
    
    assert result is not None, "Function returned None"
    assert isinstance(result, np.ndarray), "Result should be numpy array"
    assert result.shape == (2, 4), f"Expected shape (2, 4), got {result.shape}"
    
    # Check that loadings are unit vectors (approximately)
    pc1_norm = np.linalg.norm(result[0])
    pc2_norm = np.linalg.norm(result[1])
    assert abs(pc1_norm - 1.0) < 0.1, f"PC1 should be unit vector, norm = {pc1_norm:.3f}"
    assert abs(pc2_norm - 1.0) < 0.1, f"PC2 should be unit vector, norm = {pc2_norm:.3f}"
    
    # Check that PCs are orthogonal
    dot_product = np.dot(result[0], result[1])
    assert abs(dot_product) < 0.1, f"PCs should be orthogonal, dot product = {dot_product:.3f}"
    
    print("✓ Successfully extracted PCA loadings!")
    print(f"  - Shape: {result.shape}")
    print("  - PC1 loadings:", [f"{x:.3f}" for x in result[0]])
    print("  - PC2 loadings:", [f"{x:.3f}" for x in result[1]])
    
    # Interpret the loadings
    feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
    print("\n  Feature importance in PC1:")
    for i, name in enumerate(feature_names):
        print(f"    {name}: {result[0, i]:.3f}")
    
    print("\n  Feature importance in PC2:")
    for i, name in enumerate(feature_names):
        print(f"    {name}: {result[1, i]:.3f}")

validate()

## KOAN 11.6: t-SNE for Non-linear Visualization
**Objective**: Apply t-SNE for non-linear dimensionality reduction  
**Difficulty**: Intermediate-Advanced

While PCA is linear, t-SNE (t-distributed Stochastic Neighbor Embedding) can capture non-linear relationships. It's especially good for visualization, preserving local neighborhoods.

**Key Concept**: t-SNE is primarily for visualization, not feature reduction. It's computationally expensive but often reveals structure that PCA misses.

In [ ]:
def apply_tsne_to_digits():
    """
    Apply t-SNE to a subset of the digits dataset for visualization.
    
    Note: We'll use only the first 500 samples to keep computation time reasonable.
    t-SNE is much slower than PCA, especially on large datasets.
    
    Returns:
        np.ndarray: 2D t-SNE embedding, shape (500, 2)
    """
    digits = load_digits()
    
    # Use only first 500 samples for reasonable computation time
    X_subset = digits.data[:500]
    
    # TODO: Create TSNE instance with n_components=2 and random_state=42
    # Hint: Use TSNE(n_components=2, random_state=42)
    tsne = None
    
    # TODO: Fit and transform the data subset
    X_tsne = None
    
    return X_tsne

@validator.koan(6, "t-SNE for Non-linear Visualization", difficulty="Intermediate-Advanced")
def validate():
    result = apply_tsne_to_digits()
    
    assert result is not None, "Function returned None"
    assert isinstance(result, np.ndarray), "Result should be numpy array"
    assert result.shape == (500, 2), f"Expected shape (500, 2), got {result.shape}"
    
    # Check that embedding has reasonable spread
    assert np.std(result[:, 0]) > 1, "t-SNE dimension 1 should have reasonable variance"
    assert np.std(result[:, 1]) > 1, "t-SNE dimension 2 should have reasonable variance"
    
    # Check that values are not all the same
    assert len(np.unique(result[:, 0])) > 10, "t-SNE should produce diverse values"
    assert len(np.unique(result[:, 1])) > 10, "t-SNE should produce diverse values"
    
    print("✓ Successfully applied t-SNE to digits dataset!")
    print(f"  - Input shape: (500, 64)")
    print(f"  - Output shape: {result.shape}")
    print(f"  - Dimension 1 range: [{result[:, 0].min():.1f}, {result[:, 0].max():.1f}]")
    print(f"  - Dimension 2 range: [{result[:, 1].min():.1f}, {result[:, 1].max():.1f}]")
    
    print("\n  💡 Tip: t-SNE is great for visualization but:")
    print("     • Much slower than PCA")
    print("     • Not deterministic (despite random_state)")
    print("     • Distances between clusters are not meaningful")
    print("     • Primarily for visualization, not preprocessing")

validate()

## KOAN 11.7: Data Standardization Before PCA
**Objective**: Understand the importance of scaling features before PCA  
**Difficulty**: Intermediate-Advanced

PCA is sensitive to feature scales. If one feature has much larger values than others, it will dominate the principal components. Standardization ensures each feature contributes equally.

**Key Concept**: Always standardize features before PCA when they have different units or scales. Use StandardScaler to make features have mean=0 and std=1.

In [ ]:
def standardize_breast_cancer_data():
    """
    Load breast cancer dataset and standardize the features.
    
    The breast cancer dataset has 30 features with very different scales.
    Some are tiny (like concavity) while others are large (like area).
    
    Returns:
        np.ndarray: Standardized feature matrix where each feature has mean≈0, std≈1
    """
    # Load the breast cancer dataset
    cancer = load_breast_cancer()
    X = cancer.data  # Shape: (569, 30)
    
    # TODO: Create StandardScaler instance
    scaler = None
    
    # TODO: Fit the scaler and transform the data
    # Hint: Use scaler.fit_transform(X)
    X_standardized = None
    
    return X_standardized

@validator.koan(7, "Data Standardization Before PCA", difficulty="Intermediate-Advanced")
def validate():
    result = standardize_breast_cancer_data()
    
    assert result is not None, "Function returned None"
    assert isinstance(result, np.ndarray), "Result should be numpy array"
    assert result.shape == (569, 30), f"Expected shape (569, 30), got {result.shape}"
    
    # Check standardization: mean should be close to 0, std close to 1
    means = np.mean(result, axis=0)
    stds = np.std(result, axis=0, ddof=0)
    
    assert np.allclose(means, 0, atol=1e-10), f"Means should be ~0, got max={np.max(np.abs(means)):.2e}"
    assert np.allclose(stds, 1, atol=1e-10), f"Stds should be ~1, got range=[{np.min(stds):.3f}, {np.max(stds):.3f}]"
    
    print("✓ Successfully standardized breast cancer dataset!")
    print(f"  - Shape: {result.shape}")
    print(f"  - Mean range: [{np.min(means):.2e}, {np.max(means):.2e}]")
    print(f"  - Std range: [{np.min(stds):.3f}, {np.max(stds):.3f}]")
    
    # Show the impact of standardization
    cancer = load_breast_cancer()
    original_means = np.mean(cancer.data, axis=0)
    original_stds = np.std(cancer.data, axis=0, ddof=0)
    
    print(f"\n  📊 Before standardization:")
    print(f"     Mean range: [{np.min(original_means):.1f}, {np.max(original_means):.1f}]")  
    print(f"     Std range: [{np.min(original_stds):.1f}, {np.max(original_stds):.1f}]")
    print(f"  📊 After standardization:")
    print(f"     Mean range: [{np.min(means):.2e}, {np.max(means):.2e}]")
    print(f"     Std range: [{np.min(stds):.3f}, {np.max(stds):.3f}]")

validate()

## KOAN 11.8: The Curse of Dimensionality
**Objective**: Demonstrate how distance metrics behave in high dimensions  
**Difficulty**: Intermediate-Advanced

As dimensions increase, several strange things happen:
- All points become approximately equidistant
- The difference between nearest and farthest neighbors shrinks
- Most of the volume of a hypersphere is near its surface

**Key Concept**: In high dimensions, intuitions from 2D/3D often fail. This is why dimensionality reduction is crucial for many ML algorithms.

In [ ]:
def demonstrate_curse_of_dimensionality():
    """
    Compare distance distributions in low vs high dimensions.
    
    We'll generate random points in 2D vs 100D and compare how the 
    pairwise distances are distributed. In high dimensions, all distances
    become more similar (less variance).
    
    Returns:
        tuple: (std_high_d, std_low_d) - standard deviations of distances
    """
    np.random.seed(42)  # For reproducible results
    
    # Generate random points in different dimensions
    n_points = 100
    
    # TODO: Generate 100 random points in 100 dimensions (shape: 100 x 100)
    # Hint: Use np.random.rand(n_points, 100)
    points_high_d = None
    
    # TODO: Generate 100 random points in 2 dimensions (shape: 100 x 2)  
    points_low_d = None
    
    # TODO: Calculate pairwise distances for both using pdist
    # Hint: from scipy.spatial.distance import pdist (already imported)
    distances_high_d = None
    distances_low_d = None
    
    # TODO: Calculate standard deviation of distances for both
    std_high_d = None
    std_low_d = None
    
    return std_high_d, std_low_d

@validator.koan(8, "The Curse of Dimensionality", difficulty="Intermediate-Advanced") 
def validate():
    std_high_d, std_low_d = demonstrate_curse_of_dimensionality()
    
    assert std_high_d is not None and std_low_d is not None, "Function returned None values"
    assert isinstance(std_high_d, (float, np.floating)), "std_high_d should be a number"
    assert isinstance(std_low_d, (float, np.floating)), "std_low_d should be a number"
    
    # In high dimensions, distances become more uniform (lower std)
    assert std_high_d < std_low_d, f"High-D std ({std_high_d:.3f}) should be < Low-D std ({std_low_d:.3f})"
    
    # Sanity checks
    assert std_high_d > 0, "Standard deviation should be positive"
    assert std_low_d > 0, "Standard deviation should be positive"
    assert std_high_d < 1.0, "High-D std seems too large"
    assert std_low_d < 1.0, "Low-D std seems too large"
    
    print("✓ Successfully demonstrated curse of dimensionality!")
    print(f"  - 100D distance std: {std_high_d:.4f}")
    print(f"  - 2D distance std:   {std_low_d:.4f}")
    print(f"  - Ratio (100D/2D):   {std_high_d/std_low_d:.3f}")
    
    print(f"\n  🎯 Key insight: In high dimensions, all points become")
    print(f"     approximately equidistant! This makes many ML algorithms")
    print(f"     struggle because distance-based metrics become less informative.")
    
    # Show actual distance distributions
    np.random.seed(42)
    points_high = np.random.rand(100, 100)
    points_low = np.random.rand(100, 2)
    dist_high = pdist(points_high)
    dist_low = pdist(points_low)
    
    print(f"\n  📊 Distance statistics:")
    print(f"     100D: mean={np.mean(dist_high):.3f}, std={np.std(dist_high):.3f}")
    print(f"     2D:   mean={np.mean(dist_low):.3f}, std={np.std(dist_low):.3f}")

validate()

## 🎉 Congratulations!

You have mastered the fundamentals of dimensionality reduction! 

### What You've Learned
- ✅ **PCA Fundamentals**: Applied linear dimensionality reduction
- ✅ **Variance Analysis**: Understood explained variance ratios and component selection  
- ✅ **Feature Interpretation**: Analyzed PCA loadings to understand components
- ✅ **Visualization**: Used PCA and t-SNE for data visualization
- ✅ **Preprocessing**: Learned the importance of standardization
- ✅ **Theory**: Understood the curse of dimensionality

### Key Takeaways
1. **PCA** is fast, interpretable, and great for preprocessing
2. **t-SNE** excels at visualization but is computationally expensive
3. **Always standardize** when features have different scales
4. **Explained variance** helps choose the right number of components
5. **High dimensions** make distance-based algorithms struggle

### Next Steps
- **Notebook 12**: Ensemble Methods (Random Forests, Gradient Boosting)
- **Advanced**: Try UMAP, Autoencoders, or Manifold Learning
- **Practice**: Apply dimensionality reduction to your own datasets

### Real-World Applications
- **Computer Vision**: Reduce image dimensionality for faster processing
- **Genomics**: Analyze high-dimensional gene expression data  
- **Finance**: Portfolio optimization with many assets
- **NLP**: Reduce word embedding dimensions
- **Recommendation**: Collaborative filtering with matrix factorization

Keep practicing and exploring! 🚀

In [ ]:
# Final Progress Check
progress = tracker.get_notebook_progress('11_dimensionality_reduction')
print(f"\n📊 Your Progress: {progress}% complete!")

if progress == 100:
    print("🎉 Perfect! You've mastered all dimensionality reduction koans!")
    print("🎯 You're ready for Notebook 12: Ensemble Methods")
elif progress >= 75:
    print("🌟 Excellent progress! Just a few more koans to go.")
elif progress >= 50:
    print("💪 Great work! You're halfway through the challenges.")
else:
    print("🚀 Keep going! Each koan builds important skills.")

print(f"\n📈 Overall course progress:")
total_notebooks = 15
completed_notebooks = len([nb for nb in range(1, 12) if tracker.get_notebook_progress(f'{nb:02d}_*') == 100])
print(f"   Completed notebooks: {completed_notebooks}/{total_notebooks}")
print(f"   Course progress: {(completed_notebooks/total_notebooks)*100:.1f}%")